In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import pickle

In [5]:
df=pd.read_csv("model training dataset.csv")
df.head()

,Unnamed: 0,Venue,Bat First,Bat Second,runs left,balls left,wickets left,crr,rrr,result
0,0,The Rose Bowl,England,Australia,180,119,10,0.0,9.08,0
1,1,The Rose Bowl,England,Australia,176,118,10,12.0,8.95,0
2,2,The Rose Bowl,England,Australia,176,117,10,8.0,9.03,0
3,3,The Rose Bowl,England,Australia,176,116,10,6.0,9.10,0
4,4,The Rose Bowl,England,Australia,172,115,10,9.6,8.97,0


In [6]:
df.drop(columns=['Unnamed: 0'],inplace=True)
df.shape

(97737, 9)

In [7]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.8)
x.describe()

,runs left,balls left,wickets left,crr,rrr
count,97737.000000,97737.000000,97737.00000,97737.000000,97737.000000
mean,92.280661,64.286504,7.26469,7.335667,10.443846
std,49.656606,32.806349,2.30095,2.451207,13.907903
min,-18.000000,1.000000,-1.00000,0.000000,-24.000000
25%,53.000000,37.000000,6.00000,6.000000,6.740000
50%,91.000000,65.000000,8.00000,7.250000,8.710000
75%,128.000000,93.000000,9.00000,8.620000,10.970000
max,273.000000,119.000000,10.00000,60.000000,822.000000


In [8]:
ct=ColumnTransformer([("trf",OneHotEncoder(sparse=False,drop="first"),['Venue','Bat First', 'Bat Second'])],remainder="passthrough")

In [9]:
pipeline_rf=Pipeline(steps=[("step 1",ct),("step 2",RandomForestClassifier())])
pipeline_rf.fit(x_train,y_train)
y_pred_rf=pipeline_rf.predict(x_test)
print("Accuracy with Random Forest:",accuracy_score(y_test,y_pred_rf))
for i in range(20):
    print(pipeline_rf.predict_proba(x_test)[i])

Accuracy with Random Forest: 0.9870955365136207
[0.01 0.99]
[1. 0.]
[0.92 0.08]
[0.82 0.18]
[0. 1.]
[1. 0.]
[0. 1.]
[0.93 0.07]
[0.06 0.94]
[0. 1.]
[0.62 0.38]
[1. 0.]
[0.85 0.15]
[0.14 0.86]
[0.84 0.16]
[0.1 0.9]
[0.84 0.16]
[0.99 0.01]
[0.3 0.7]
[0.98 0.02]


In [10]:
pipeline_lr=Pipeline(steps=[("step 1",ct),("step 2",LogisticRegression(solver="liblinear"))])
pipeline_lr.fit(x_train,y_train)
y_pred_lr=pipeline_lr.predict(x_test)
print("Accuracy with Logistic regression:",accuracy_score(y_test,y_pred_lr))
for i in range(20):
    print(pipeline_lr.predict_proba(x_test)[i])

Accuracy with Logistic regression: 0.8595600460416933
[0.27595804 0.72404196]
[0.99834609 0.00165391]
[0.74210643 0.25789357]
[0.31732796 0.68267204]
[0.044241 0.955759]
[0.99891307 0.00108693]
[0.02816439 0.97183561]
[0.92585426 0.07414574]
[0.43126725 0.56873275]
[0.00383519 0.99616481]
[0.28639519 0.71360481]
[9.99994680e-01 5.32014574e-06]
[0.5953367 0.4046633]
[0.24072017 0.75927983]
[0.91717662 0.08282338]
[0.04998857 0.95001143]
[0.7924993 0.2075007]
[0.89200379 0.10799621]
[0.23838801 0.76161199]
[0.55550806 0.44449194]


In [16]:
pickle.dump(pipeline_lr,open("lr_pipeline.pkl","wb"))

In [17]:
pickle.dump(pipeline_rf,open("rf_pipeline.pkl","wb"))

In [12]:
# trf_x_train=ct.fit_transform(x_train)
# trf_x_test=ct.fit_transform(x_test)
# params={"C":[0.01,0.1,1,10,100],"kernel":["linear","sigmoid","rbf"]}
# grid=GridSearchCV(SVC(),params)
# grid.fit(trf_x_train,y_train)

In [13]:
# print(grid.best_estimator_)
# print(grid.score(trf_x_test,y_test))

In [14]:
# trf_x_train=ct.fit_transform(x_train)
# trf_x_test=ct.fit_transform(x_test)
# C=[10,100]
# for c in tqdm(C):
#     model=SVC(kernel='linear',C=c)
#     model.fit(trf_x_train,y_train)
#     y_pred=model.predict(trf_x_test)
#     print(f"accuracy with linear kernel and c={C} is",accuracy_score(y_test,y_pred))